In [2]:

%load_ext autoreload 
# %aimport rl_envs.grid_world_env

%autoreload 2
import torch
import math
from torch.utils.tensorboard import SummaryWriter # type: ignore

from agents.A2C import A2CAgent
from tools.helper import *
import  gymnasium  as gym
from rl_envs.new_gym_grid_world_env import GridWorldEnv
from torch.nn import functional as F
from collections import defaultdict
import itertools
from stable_baselines3 import A2C
%autoreload 2


pygame 2.5.2 (SDL 2.28.3, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
DISCOUNTED_FACTOR = 0.999
FORBIDDEN_REWARD = -10
HITWALL_REWARD = -10
TARGET_REWARD = 10
NORMAL_REWARD = -1

SEED = 666
torch.manual_seed(SEED)


In [4]:
# env = GridWorldEnv(size=3, fixed_map = True, forbidden_grids=[(1,1),(1,2)], target_grids=[(2,2)], forbidden_reward=FORBIDDEN_REWARD, hit_wall_reward=HITWALL_REWARD, target_reward=TARGET_REWARD, normal_reward=NORMAL_REWARD)
env = GridWorldEnv(fixed_map = True, forbidden_grids=[(1,1),(1,2), (2,2),(3,1),(3,3),(4,1)], target_grids=[(3,2)], forbidden_reward=FORBIDDEN_REWARD, hit_wall_reward=HITWALL_REWARD, target_reward=TARGET_REWARD, normal_reward=NORMAL_REWARD)
agent = A2CAgent(2, 5, height= env.height, width=env.width, discounted_factor=DISCOUNTED_FACTOR)


In [5]:

writer = SummaryWriter()
num_episodes = 100000
episode_len = 100
trajectory = []
obs, _ = env.reset()
episode_rewards = defaultdict(float)
TD_step = 1 # perform TD(5)
running_reward = -10
for i_episode  in range(num_episodes):
    # 首先, 根据 policy 生成 episode
    terminated = truncated = False
    next_obs, _ = env.reset()
    episode_record = []
    # del agent.saved_log_probs[:]

    for _ in range(TD_step-1):
        # del agent.saved_log_prob
        obs = next_obs
        action = agent.get_action(obs) # action 这里也有随机性
        next_obs, reward, terminated, truncated, info = env.step(action)
        if reward == FORBIDDEN_REWARD:
            truncated = True
        episode_record.append((obs, action, reward, next_obs))
        
        if terminated or truncated:
            break

    for t in range(episode_len):
        if not (terminated or truncated):
            obs = next_obs # 注意这里的 state 要和 之前环境里的 agent 的位置对上 (TD queue中最后一个), 不要被下面计算用的 state 干扰
            action = agent.get_action(obs) # action 这里也有随机性
            next_obs, reward, terminated, truncated, info = env.step(action)
            if reward == FORBIDDEN_REWARD:
                truncated = True
            episode_record.append((obs, action, reward, next_obs))

        if len(episode_record) == 0:
            break

        # if not (terminated or truncated):
            # with torch.no_grad():
        discounted_reward = agent.value_net(next_obs) 
        # else: # next state is the end, although maybe I should still use v estimate
        #     discounted_reward = torch.tensor(0.)
        for i, (state, action, reward, next_state) in enumerate(episode_record[::-1]):
            discounted_reward = reward + agent.discounted_factor * discounted_reward

        # discounted_reward = sum(agent.discounted_factor**i * t[2] for i, t in enumerate(episode_record[t:]))
        state, action, reward, next_state = episode_record.pop(0)

        # total_return = sum(agent.discounted_factor**i * t.reward for i, t in enumerate(episode_record[t:]))

        # value_next = agent.value_net(next_state)
        # with torch.no_grad():
        v_value = agent.value_net(state).squeeze() 

        # TD_target = reward + agent.discounted_factor * value_next
        TD_target = discounted_reward.squeeze() 
        TD_error = TD_target - v_value # TD_error is the advantage (每一处加减, 都要注意 broadcasting)

        # Update statistics
        episode_rewards[i_episode] += reward
        # with torch.no_grad():
        action_probs = agent.policy_net(state)
        action_prob = action_probs[0][action]

        agent.optimizer_v.zero_grad()
        loss2 =  F.mse_loss(TD_target, v_value)
        loss2.sum().backward(retain_graph=True)
        agent.optimizer_v.step()

        agent.optimizer.zero_grad()
        # loss = -agent.saved_log_prob * TD_error
        loss = -action_prob * TD_error
        loss.sum().backward()
        # torch.nn.utils.clip_grad.clip_grad_norm_(agent.policy_net.parameters(), 100)
        agent.optimizer.step()
    running_reward = 0.05*episode_rewards[i_episode] + 0.95*running_reward
    if i_episode % 100 == 1:
        print("\repisode_len {} @ Episode {}/{} ({}, {}, {})".format(
                t, i_episode, num_episodes, episode_rewards[i_episode], episode_rewards[i_episode - 1], running_reward))

        # if terminated or truncated:
        #     break
print("end train")

episode_len 1 @ Episode 1/100000 (-10.0, -12.0, -10.094999999999999)
episode_len 2 @ Episode 101/100000 (9.0, -10.0, -8.788549005803228)
episode_len 1 @ Episode 201/100000 (-10.0, -11.0, -10.89062863136899)
episode_len 12 @ Episode 301/100000 (-21.0, -13.0, -11.450130452851505)
episode_len 1 @ Episode 401/100000 (-10.0, -10.0, -11.355111468559514)
episode_len 6 @ Episode 501/100000 (-15.0, -16.0, -11.836107482133398)
episode_len 2 @ Episode 601/100000 (-11.0, -15.0, -11.98984723937532)
episode_len 6 @ Episode 701/100000 (-15.0, -10.0, -12.316488281054534)
episode_len 6 @ Episode 801/100000 (-15.0, -17.0, -11.676342931376125)
episode_len 1 @ Episode 901/100000 (-10.0, -12.0, -11.946577630556089)
episode_len 1 @ Episode 1001/100000 (-10.0, -11.0, -12.235638554885599)
episode_len 3 @ Episode 1101/100000 (8.0, -11.0, -14.024030846177146)
episode_len 3 @ Episode 1201/100000 (-12.0, -30.0, -12.826787183620764)
episode_len 2 @ Episode 1301/100000 (-11.0, -12.0, -14.247234169316107)
episode_le

In [7]:
# visualize_in_gym(agent, "CartPole-v1")
policy = agent.generate_policy_table(env.height, env.width)
v = agent.generate_v_table(env.height, env.width)

print(env)
print_by_dict(env, policy)
print()
print_by_dict(env, v)

for i in range(env.height):
    print("[", end=" ")
    for j in range(env.width):
        state = (i,j)
        action = np.argmax(policy[state])
        print(env.action_mappings[action], end=" ")
    print("]")

[ 0.00 0.00 0.00 0.00 0.00 ]
[ 0.00 -10.00 -10.00 0.00 0.00 ]
[ 0.00 0.00 -10.00 0.00 0.00 ]
[ 0.00 -10.00 10.00 -10.00 0.00 ]
[ 0.00 -10.00 0.00 0.00 0.00 ]

[ [[0.02167876 0.9332449  0.01043832 0.01374269 0.02089541]] [[0.00577145 0.9580991  0.00533615 0.01408157 0.01671168]] [[0.0074049  0.9775146  0.0070077  0.00429381 0.00377895]] [[0.8956004  0.09499405 0.00152254 0.00338234 0.0045006 ]] [[0.96795684 0.00418292 0.00533816 0.01056109 0.01196107]] ]
[ [[0.02295563 0.01588685 0.9358886  0.00570277 0.01956627]] [[0.13948002 0.0415951  0.6747937  0.11467199 0.02945918]] [[0.08761629 0.73658025 0.10467259 0.04673047 0.02440037]] [[0.6253132  0.3392152  0.01222425 0.00558564 0.01766176]] [[0.97023106 0.0062426  0.01178719 0.00483981 0.00689937]] ]
[ [[0.28497866 0.30371732 0.3391284  0.02151516 0.05066053]] [[0.7966191  0.13968658 0.01288569 0.0221806  0.02862793]] [[0.9634684  0.01541861 0.01186214 0.00722571 0.00202509]] [[0.02629561 0.94285774 0.01252797 0.00971116 0.00860762]] [[0.9

: 